In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from IPython.display import display
import pandas as pd
import h5py
import os
# to load data from the test file
from utils.create_datasets import SumDatasets
# To load the data sets here, also we will create some examples to explore torch
data_dir = '../data/'
from utils import config

/home/chenxiang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sum_dataset = SumDatasets(os.path.join(data_dir, 'features-600-40_v2.hdf5'))

In [3]:
example_1 = sum_dataset[0:20] # assume we have a batch data of 20 items.
original_data = pd.read_csv(os.path.join(data_dir, 'test_data.csv'), encoding='utf-8')
original_data['content_len'] = original_data.content.apply(lambda x: len(x.split()))
display(original_data.head(n=20))
display(example_1[0][0])

,id,content,title,content_len
0,1,This was the year that felt like 50 years. We ...,21 Stories Our Readers Loved in 2017,4142
1,2,Gary Vaynerchuk once told a 20 year old Taylor...,What To Do After Graduating College,2054
2,3,"On this episode of Recode Decode, hosted by Ka...",Full transcript: Former Groupon CEO Andrew Mas...,11679
3,4,Today we are going to talk about the Albigensi...,The Albigensian Crusade And The Black Mass,992
4,5,IntroductionThe more you want to impact the re...,The Surprising Secret To Changing The World,5153
5,6,I began my short term (sometimes business rela...,Five Things I’ve Learned from Short-Term Trave...,866
6,7,Either I’ve become way better at choosing whic...,Influential books I read in the last 2 years,2993
7,8,You can feel it: 2018 is the year you finally ...,"How to spend less, save more, and clean up you...",1617
8,9,U.S. Soccer Federation president Sunil Gulati ...,Face of US Soccer since 2006 wants out,421
9,10,"When things get tough or simply flat, many peo...",Leaving Google for Friendz: how we did handle ...,1277


array([   18,    20,     1,    55,     8,   877,    60,   437,    99,
          25,   589,  1570,    25,  7430,    44, 36586,    25,  7583,
        2610,   358,     3,  2019,    25,   183,   420,    25,    85,
           2,   202,   287,    23,     1,   139,   229,     3,   335,
          25,   371,   236,  2070,  2532,     2,   602, 10228,    25,
         877,  4903,   143,    27,   231,   726,  4903,    33,    42,
           1,   100,    25, 12693,    25, 14674,    25,  2137,  1460,
          44,    13,     1,   179,   201, 50001,    31,    50,  2911,
          21,   848,   815,    49,     4,     1,  1236,    25,   757,
           6,  3655,     3,   367,     8,  1149, 17300,    13,    68,
        2739,    77,    94,   306,   197,   332,    32,    84,   815,
          43,   273,   758,  1887,   905,   358,  1650,   246,   532,
          32,    84,   815,    43,    71,    25,  1137,  2070,     2,
           1,   600,   104,     3,   815,    43,    46,    25,    21,
         471,  1580,

In [7]:
embedding_dim = 50
hidden_dim = 10
# for the padding index 0 and the Unktoken vocab_size + 1, so there will be NUM_WORDS+2
# tokens
embedding = nn.Embedding(config.NUM_WORDS +2, embedding_dim=embedding_dim)
lstm = nn.LSTM(embedding_dim, hidden_dim)

In [8]:
x = torch.from_numpy(example_1[0])

In [18]:
embeded_x = embedding(x)
output, hidden = lstm(embeded_x)
h = output.contiguous()
display(h.shape)
max_h, _ = h.max(dim=1) # find each batch's maximum hidden state
display(max_h)

torch.Size([20, 600, 10])

tensor([[0.5158, 0.5986, 0.5767, 0.6330, 0.6154, 0.5766, 0.5469, 0.5183, 0.5927,
         0.6202],
        [0.7223, 0.7004, 0.7794, 0.7969, 0.6252, 0.6994, 0.6700, 0.5775, 0.7517,
         0.5574],
        [0.7473, 0.7252, 0.7682, 0.7023, 0.5624, 0.8189, 0.6409, 0.5131, 0.7309,
         0.4857],
        [0.5584, 0.6599, 0.6808, 0.7636, 0.5259, 0.7098, 0.5762, 0.4847, 0.6862,
         0.6399],
        [0.6476, 0.7076, 0.5770, 0.7636, 0.6496, 0.6973, 0.7276, 0.4431, 0.7674,
         0.7345],
        [0.6452, 0.7501, 0.7968, 0.8239, 0.5946, 0.7437, 0.7452, 0.5760, 0.7495,
         0.7888],
        [0.6735, 0.7684, 0.7626, 0.8101, 0.5257, 0.6962, 0.7921, 0.6199, 0.6625,
         0.6860],
        [0.6611, 0.7546, 0.9221, 0.8342, 0.5152, 0.5993, 0.5278, 0.5426, 0.7551,
         0.7246],
        [0.6412, 0.7144, 0.7900, 0.6860, 0.5302, 0.7327, 0.5842, 0.4725, 0.6623,
         0.7129],
        [0.6147, 0.6282, 0.7176, 0.7142, 0.7873, 0.7952, 0.7722, 0.4929, 0.7931,
         0.7722],
        [0

In [13]:
h_c, h_ = hidden

In [19]:
h_c.unsqueeze(0).shape

torch.Size([1, 1, 600, 10])